# Homework 01 - Frequent Pattern Mining

Name: Dylan Zucker & Dempsey Wade 

Class: CSCI 349 - Intro to Data Mining

Semester: 2019SP

Instructor: Brian King

# Association Rule Mining

Finding patterns in movie ratings data to reccomend movies to people based on their preference history. 

In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

## Part 2  - Applying Frequent Pattern Mining: MovieLens

### Data setup and Exploration

In [2]:
# Read in Data:
df_ratings = pd.read_csv("../data/ml-latest-small/ratings.csv")
df_tags = pd.read_csv("../data/ml-latest-small/tags.csv")
df_links = pd.read_csv("../data/ml-latest-small/links.csv")
df_movies = pd.read_csv("../data/ml-latest-small/movies.csv")
df_movies.set_index("movieId", inplace = True)

In [3]:
# Show data
display(df_ratings.head())
display(df_tags.head())
display(df_links.head())
display(df_movies.head())

userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931

userId  movieId              tag   timestamp
0       2    60756            funny  1445714994
1       2    60756  Highly quotable  1445714996
2       2    60756     will ferrell  1445714992
3       2    89774     Boxing story  1445715207
4       2    89774              MMA  1445715200

movieId  imdbId   tmdbId
0        1  114709    862.0
1        2  113497   8844.0
2        3  113228  15602.0
3        4  114885  31357.0
4        5  113041  11862.0

title  \
movieId                                       
1                          Toy Story (1995)   
2                            Jumanji (1995)   
3                   Grumpier Old Men (1995)   
4                  Waiting to Exhale (1995)   
5        Father of the Bride Part II (1995)   

                                              genres  
movieId                                               
1        Adventure|Animation|Children|Comedy|Fantasy  
2                         Adventure|Children|Fantasy  
3                                     Comedy|Romance  
4                               Comedy|Drama|Romance  
5                                             Comedy

In [4]:
# Check info and describe
display(df_ratings.info())
display(df_ratings.describe())
display(df_tags.info())
display(df_tags.describe())
display(df_links.info())
display(df_links.describe())
display(df_movies.info())
display(df_movies.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
userId       100836 non-null int64
movieId      100836 non-null int64
rating       100836 non-null float64
timestamp    100836 non-null int64
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


None

userId        movieId         rating     timestamp
count  100836.000000  100836.000000  100836.000000  1.008360e+05
mean      326.127564   19435.295718       3.501557  1.205946e+09
std       182.618491   35530.987199       1.042529  2.162610e+08
min         1.000000       1.000000       0.500000  8.281246e+08
25%       177.000000    1199.000000       3.000000  1.019124e+09
50%       325.000000    2991.000000       3.500000  1.186087e+09
75%       477.000000    8122.000000       4.000000  1.435994e+09
max       610.000000  193609.000000       5.000000  1.537799e+09

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
userId       3683 non-null int64
movieId      3683 non-null int64
tag          3683 non-null object
timestamp    3683 non-null int64
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


None

...

None

movieId        imdbId         tmdbId
count    9742.000000  9.742000e+03    9734.000000
mean    42200.353623  6.771839e+05   55162.123793
std     52160.494854  1.107228e+06   93653.481487
min         1.000000  4.170000e+02       2.000000
25%      3248.250000  9.518075e+04    9665.500000
50%      7300.000000  1.672605e+05   16529.000000
75%     76232.000000  8.055685e+05   44205.750000
max    193609.000000  8.391976e+06  525662.000000

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9742 entries, 1 to 193609
Data columns (total 2 columns):
title     9742 non-null object
genres    9742 non-null object
dtypes: object(2)
memory usage: 228.3+ KB


None

title genres
count          9742   9742
unique         9737    951
top     Emma (1996)  Drama
freq              2   1053

### Finding Association Rules For Movies Watched

### Binarize

In [5]:
# Finding frequent item sets for movies watched in df_ratings
# Binarize the movies based on movies and group by the user
df_bin_ratings_movies = pd.get_dummies(df_ratings, columns=["movieId"]).groupby("userId").sum().apply(lambda y: y.apply(lambda x : x if x == 0 else 1), axis = 1)
df_bin_ratings_movies.head()

rating  timestamp  movieId_1  movieId_2  movieId_3  movieId_4  \
userId                                                                  
1          1.0        1.0        1.0        0.0        1.0        0.0   
2          1.0        1.0        0.0        0.0        0.0        0.0   
3          1.0        1.0        0.0        0.0        0.0        0.0   
4          1.0        1.0        0.0        0.0        0.0        0.0   
5          1.0        1.0        1.0        0.0        0.0        0.0   

        movieId_5  movieId_6  movieId_7  movieId_8       ...        \
userId                                                   ...         
1             0.0        1.0        0.0        0.0       ...         
2             0.0        0.0        0.0        0.0       ...         
3             0.0        0.0        0.0        0.0       ...         
4             0.0        0.0        0.0        0.0       ...         
5             0.0        0.0        0.0        0.0       ...         

        movieId_193565  movieId_193567  movieId_193571  movieId_193573  \
userId                                                                   
1                  0.0             0.0             0.0             0.0   
2                  0.0             0.0             0.0             0.0   
3                  0.0             0.0             0.0             0.0   
4                  0.0             0.0             0.0             0.0   
5                  0.0             0.0             0.0             0.0   

        movieId_193579  movieId_193581  movieId_193583  movieId_193585  \
userId                                                                   
1                  0.0             0.0             0.0             0.0   
2                  0.0             0.0             0.0             0.0   
3                  0.0             0.0             0.0             0.0   
4                  0.0             0.0             0.0             0.0   
5                  0.0             0.0             0.0             0.0   

        movieId_193587  movieId_193609  
userId                                  
1                  0.0             0.0  
2                  0.0             0.0  
3                  0.0             0.0  
4                  0.0             0.0  
5                  0.0             0.0  

[5 rows x 9726 columns]

### Find Apriori

In [6]:
# Get the apriori
ap_ratings_movies = apriori(df_bin_ratings_movies.drop(["rating", "timestamp"], axis = 1), min_support=0.15, use_colnames=True)
ap_ratings_movies.shape[0]

3014

### Find Association Rules 

In [7]:
# Get association Rules
ratings_movies_assoc_rules = association_rules(ap_ratings_movies, metric="support", min_threshold=.01)
ratings_movies_assoc_rules.shape[0]

19578

### View Most Confident Assocation Rules

In [8]:
ratings_movies_assoc_rules.sort_values(by="confidence", ascending=False).head(10)

antecedents     consequents  \
3245                          (movieId_110, movieId_500)   (movieId_356)   
11437          (movieId_1198, movieId_1210, movieId_296)   (movieId_260)   
17867  (movieId_1196, movieId_356, movieId_1198, movi...   (movieId_260)   
12024          (movieId_356, movieId_1198, movieId_1210)   (movieId_260)   
11619          (movieId_1196, movieId_1198, movieId_318)   (movieId_260)   
17836  (movieId_1196, movieId_356, movieId_260, movie...  (movieId_1210)   
18226  (movieId_1196, movieId_1270, movieId_1198, mov...   (movieId_260)   
13270         (movieId_1270, movieId_1198, movieId_1210)   (movieId_260)   
12836         (movieId_1196, movieId_1270, movieId_1210)   (movieId_260)   
13298         (movieId_2571, movieId_1198, movieId_1210)   (movieId_260)   

       antecedent support  consequent support   support  confidence      lift  \
3245             0.150820            0.539344  0.150820    1.000000  1.854103   
11437            0.150820            0.411475  0.150820    1.000000  2.430279   
17867            0.160656            0.411475  0.160656    1.000000  2.430279   
12024            0.162295            0.411475  0.162295    1.000000  2.430279   
11619            0.159016            0.411475  0.159016    1.000000  2.430279   
17836            0.155738            0.321311  0.155738    1.000000  3.112245   
18226            0.155738            0.411475  0.155738    1.000000  2.430279   
13270            0.157377            0.411475  0.157377    1.000000  2.430279   
12836            0.183607            0.411475  0.181967    0.991071  2.408580   
13298            0.180328            0.411475  0.178689    0.990909  2.408185   

       leverage  conviction  
3245   0.069476         inf  
11437  0.088761         inf  
17867  0.094550         inf  
12024  0.095515         inf  
11619  0.093585         inf  
17836  0.105697         inf  
18226  0.091655         inf  
13270  0.092620         inf  
12836  0.106418   65.914754  
13298  0.104488   64.737705

### Make Function to Output Assocation Rules Neatly

In [ ]:
# Function to print top n assoc rules by confidence
# Input is association rules dataframe from ratings and n
def print_top_assoc_movies(df_assoc,sort_by, n):
    print("Finding top " + str(n) + " association rules by " + sort_by + ":")
    print()
    
    df = df_assoc.sort_values(by=sort_by, ascending=False)
    df.reset_index(inplace=True)
    antecedents = df["antecedents"]
    consequents = df["consequents"]
    for i in range(n):
        ans = antecedents[i]
        cons = consequents[i]
        s = "["
        for an in ans:
            s += df_movies.loc[int(an[8:])]["title"] + ", "
            
        s = s[:len(s)-2] + "] --> ["
        for con in cons:
            s += df_movies.loc[int(con[8:])]["title"] + ", "
        s = s[:len(s)-2] + "]"
        print(s)
        print("With support of " + str(df.loc[i]["support"]) + " and confidence " + str(str(df.loc[i]["confidence"])))
        print()
    

### Call Function on using different variables

In [ ]:
print_top_assoc_movies(ratings_movies_assoc_rules, "confidence", 7)

Finding top 7 association rules by confidence:

[Braveheart (1995), Mrs. Doubtfire (1993)] --> [Forrest Gump (1994)]
With support of 0.15081967213114755 and confidence 1.0

[Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981), Star Wars: Episode VI - Return of the Jedi (1983), Pulp Fiction (1994)] --> [Star Wars: Episode IV - A New Hope (1977)]
With support of 0.15081967213114755 and confidence 1.0

[Star Wars: Episode V - The Empire Strikes Back (1980), Forrest Gump (1994), Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981), Star Wars: Episode VI - Return of the Jedi (1983)] --> [Star Wars: Episode IV - A New Hope (1977)]
With support of 0.16065573770491803 and confidence 1.0

[Forrest Gump (1994), Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981), Star Wars: Episode VI - Return of the Jedi (1983)] --> [Star Wars: Episode IV - A New Hope (1977)]
With support of 0.16229508196721312 and confidence 1.0

[

In [ ]:
print_top_assoc_movies(ratings_movies_assoc_rules, "lift", 7)

Finding top 7 association rules by lift:

[Matrix, The (1999), Star Wars: Episode IV - A New Hope (1977), Lord of the Rings: The Two Towers, The (2002)] --> [Star Wars: Episode V - The Empire Strikes Back (1980), Lord of the Rings: The Fellowship of the Ring, The (2001), Lord of the Rings: The Return of the King, The (2003)]
With support of 0.1540983606557377 and confidence 0.8468468468468467

[Star Wars: Episode V - The Empire Strikes Back (1980), Lord of the Rings: The Fellowship of the Ring, The (2001), Lord of the Rings: The Return of the King, The (2003)] --> [Matrix, The (1999), Star Wars: Episode IV - A New Hope (1977), Lord of the Rings: The Two Towers, The (2002)]
With support of 0.1540983606557377 and confidence 0.8867924528301887

[Star Wars: Episode V - The Empire Strikes Back (1980), Lord of the Rings: The Return of the King, The (2003)] --> [Matrix, The (1999), Lord of the Rings: The Fellowship of the Ring, The (2001), Star Wars: Episode IV - A New Hope (1977), Lord of th

In [ ]:
print_top_assoc_movies(ratings_movies_assoc_rules, "support", 7)

Finding top 7 association rules by support:

[Shawshank Redemption, The (1994)] --> [Forrest Gump (1994)]
With support of 0.37868852459016394 and confidence 0.7287066246056783

[Forrest Gump (1994)] --> [Shawshank Redemption, The (1994)]
With support of 0.37868852459016394 and confidence 0.7021276595744682

[Pulp Fiction (1994)] --> [Forrest Gump (1994)]
With support of 0.3770491803278688 and confidence 0.749185667752443

[Forrest Gump (1994)] --> [Pulp Fiction (1994)]
With support of 0.3770491803278688 and confidence 0.6990881458966566

[Pulp Fiction (1994)] --> [Shawshank Redemption, The (1994)]
With support of 0.3639344262295082 and confidence 0.723127035830619

[Shawshank Redemption, The (1994)] --> [Pulp Fiction (1994)]
With support of 0.3639344262295082 and confidence 0.7003154574132493

[Pulp Fiction (1994)] --> [Silence of the Lambs, The (1991)]
With support of 0.33934426229508197 and confidence 0.6742671009771988



### Function to find movie suggestions

In [ ]:
def find_n_suggestions(movie_title, df_movies, df_assoc_rules, n):
    # Get movie ID
    movieId = df_movies[df_movies["title"] == movie_title].index
    sorted_rules = df_assoc_rules.sort_values(by="support", ascending = False)
    sorted_rules.reset_index(inplace=True)
    
    rows_of_interest = sorted_rules[sorted_rules["antecedents"].str.contains("movieId_"+str(movieId[0]), regex=False)]
    rows_of_interest.reset_index(inplace=True)

    print("If you like: " + movie_title + " we think youd love...")
    print()
    if rows_of_interest.shape[0] < n:
        n = rows_of_interest.shape[0]
    for i in range(n):
        s = ""
        cons = rows_of_interest["consequents"].loc[i]
        for con in cons:
            s += df_movies.loc[int(con[8:])]["title"] + ", "
        print(s)
        print()
    
    

### Find Recomendations!!! 

In [ ]:
find_n_suggestions("Matrix, The (1999)", df_movies, ratings_movies_assoc_rules, 8)

If you like: Matrix, The (1999) we think youd love...

Forrest Gump (1994), 

Star Wars: Episode IV - A New Hope (1977), 

Shawshank Redemption, The (1994), 

Fight Club (1999), 

Pulp Fiction (1994), 

Star Wars: Episode V - The Empire Strikes Back (1980), 

Star Wars: Episode IV - A New Hope (1977), 

Star Wars: Episode V - The Empire Strikes Back (1980), Star Wars: Episode IV - A New Hope (1977), 



In [ ]:
find_n_suggestions("Godfather, The (1972)", df_movies, ratings_movies_assoc_rules, 8)

If you like: Godfather, The (1972) we think youd love...

Shawshank Redemption, The (1994), 

Matrix, The (1999), 

Pulp Fiction (1994), 

Star Wars: Episode IV - A New Hope (1977), 

Forrest Gump (1994), 

Silence of the Lambs, The (1991), 

Godfather: Part II, The (1974), 

Fight Club (1999), 



### Finding Associations for Good Movies Only

In [ ]:
df_ratings_good = df_ratings[df_ratings["rating"] >= 3]
df_bin_good_ratings_movies = pd.get_dummies(df_ratings_good, columns=["movieId"]).groupby("userId").sum().apply(lambda y: y.apply(lambda x : x if x == 0 else 1), axis = 1)
ap_good_ratings_movies = apriori(df_bin_good_ratings_movies.drop(["rating", "timestamp"], axis = 1), min_support=0.15, use_colnames=True)
good_ratings_movies_assoc_rules = association_rules(ap_good_ratings_movies, metric="support", min_threshold=.01)
good_ratings_movies_assoc_rules.shape[0]

### Call functions again with good ratings rules only

In [ ]:
print_top_assoc_movies(good_ratings_movies_assoc_rules, "confidence", 5)

In [ ]:
print_top_assoc_movies(good_ratings_movies_assoc_rules, "support", 4)

In [ ]:
find_n_suggestions("Matrix, The (1999)", df_movies, good_ratings_movies_assoc_rules, 4)

In [ ]:
find_n_suggestions("Godfather, The (1972)", df_movies, ratings_movies_assoc_rules, 4)